#### This notebook was created to perform some datascreening on the raw dataset 

In [103]:
import os 
import numpy as np
import pandas as pd 

In [16]:
%matplotlib inline

#### Read the file 

In [17]:
df = pd.read_csv('./data/HiddenFraudulentURLs.csv', sep=';')

In [59]:
df.head()

,url,compromissionType,isHiddenFraudulent,contentLength,serverType,poweredBy,contentType,lastModified
0,http://www.sinduscongoias.com.br/index.html,defacement,False,2474,Apache/2.2,NaN,text/html,"Sat, 05 Jan 2013 19:36:29 GMT"
1,http://www.sinduscongoias.com.br/index.php/ins...,defacement,False,0,Apache/2.2,NaN,text/html; charset=utf-8,"Mon, 21 Jan 2013 19:30:53 GMT"
2,http://www.sinduscongoias.com.br/index.php/ins...,defacement,False,0,Apache/2.2,NaN,text/html; charset=utf-8,"Mon, 21 Jan 2013 19:30:58 GMT"
3,http://www.sinduscongoias.com.br/index.php/ins...,defacement,False,0,Apache/2.2,NaN,text/html; charset=utf-8,"Mon, 21 Jan 2013 19:31:01 GMT"
4,http://www.sinduscongoias.com.br/index.php/ins...,defacement,False,0,Apache/2.2,NaN,text/html; charset=utf-8,"Mon, 21 Jan 2013 19:31:05 GMT"


There are 5 features :
- contentLength
- serverType
- poweredBy
- contentType
- lastModified

and 2 possible targets : 
- compromissionType
- isHiddenFraudulent

#### Statistics on targets

In [60]:
df.groupby(['compromissionType']).url.count()

compromissionType
defacement    96514
normal         3713
phishing      84953
Name: url, dtype: int64

In [61]:
df.groupby(['isHiddenFraudulent']).url.count()

isHiddenFraudulent
False    176471
True       8709
Name: url, dtype: int64

#### Remove rows duplicate

In [62]:
"Raw dataset length : {}".format(df.shape[0])

'Raw dataset length : 185180'

In [63]:
#duplicated rows
df_rows = df.loc[~df.duplicated()]
"Dataset without exact row duplicates length : {}".format(df_rows.shape[0])

'Dataset without exact row duplicates length : 142802'

There are ~40k EXACT row duplicates...  
What about url duplicates ? How many are there ?
Is it worth keeping them ?

In [64]:
#Check url duplicates 
df_rows.groupby(['url']).compromissionType.count().sort_values(ascending=False).head()

url
http://pastehtml.com/auth/twitter                                   527
http://pastehtml.com/auth/facebook                                   45
http://blog.nabatakoutdoors.com/wp-login.php?action=lostpassword     16
http://blog.nabatakoutdoors.com/wp-login.php                         16
http://otelaydinlatma.com/index.html                                  8
Name: compromissionType, dtype: int64

#### Check duplicates of single url

In [65]:
path = 'http://pastehtml.com/auth/facebook'#Paste URL here
ex = df_rows.loc[df_rows.url.str.contains(path)]  #This filters the dataframe with the given URL 
ex.head()

,url,compromissionType,isHiddenFraudulent,contentLength,serverType,poweredBy,contentType,lastModified
106795,http://pastehtml.com/auth/facebook,phishing,False,41203,NaN,NaN,text/html; charset=utf-8,NaN
160322,http://pastehtml.com/auth/facebook,phishing,False,41206,NaN,NaN,text/html; charset=utf-8,NaN
165430,http://pastehtml.com/auth/facebook,phishing,False,40538,NaN,NaN,text/html; charset=utf-8,NaN
166635,http://pastehtml.com/auth/facebook,phishing,False,40495,NaN,NaN,text/html; charset=utf-8,NaN
166661,http://pastehtml.com/auth/facebook,phishing,False,40542,NaN,NaN,text/html; charset=utf-8,NaN


In [66]:
ex.describe()

,contentLength
count,45.000000
mean,41004.511111
std,410.532781
min,40461.000000
25%,40650.000000
50%,41126.000000
75%,41320.000000
max,42192.000000


Through analysis of several url duplicates, we concluded that they only differ slightly for the contentLength. We believe it's therefore not worth keeping

#### Remove url duplicates

In [67]:
df_unique_url = df.loc[~df.duplicated(subset=['url'])]

In [68]:
"Dataset without url duplicates length : {}".format(df_dupliless.shape[0])

'Dataset without url duplicates length : 139495'

#### Distribution of samples by compromissionType/isHiddenFraudulent which are our potential targets

In [69]:
#Raw dataset
df.groupby(['compromissionType','isHiddenFraudulent']).url.count().reset_index()

,compromissionType,isHiddenFraudulent,url
0,defacement,False,94370
1,defacement,True,2144
2,normal,False,3713
3,phishing,False,78388
4,phishing,True,6565


In [70]:
#### Unique URL
df_unique_url.groupby(['compromissionType','isHiddenFraudulent']).url.count().reset_index()

,compromissionType,isHiddenFraudulent,url
0,defacement,False,93244
1,defacement,True,2121
2,normal,False,3637
3,phishing,False,33964
4,phishing,True,6529


We notice that the majority of the duplicates came from phishing websites. 

#### Example of analysis of one feature : contentLength

In [73]:
df_unique_url.contentLength.describe()

count    1.394950e+05
mean     3.924065e+03
std      2.085518e+04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.843260e+06
Name: contentLength, dtype: float64

In [71]:
df_unique_url.contentLength.isnull().unique() #check for nan values

array([False])

In [100]:
cl_0 = df_unique_url.loc[df_unique_url.contentLength == 0].groupby(['isHiddenFraudulent']).url.count()
cl_0

isHiddenFraudulent
False    113290
True       4106
Name: url, dtype: int64

In [126]:
perc_cl_0 = np.round((cl_0[0]+cl_0[1])/df_unique_url.shape[0]*100,2)
perc_fraud= np.round(cl_0[1]/cl_0[0]*100,2)

msg = "{} % of the dataset has a contentLength=0\n{} % has the label isHiddenFraudulent"
print(msg.format(perc_cl_0, perc_fraud))

84.16 % of the dataset has a contentLength=0
3.62 % has the label isHiddenFraudulent


In [127]:
cl_sup_0 = df_unique_url.loc[df_unique_url.contentLength > 0].groupby(['isHiddenFraudulent']).url.count()
cl_sup_0

isHiddenFraudulent
False    17555
True      4544
Name: url, dtype: int64

In [128]:
perc_cl_sup_0 = np.round((cl_sup_0[0]+cl_sup_0[1])/df_unique_url.shape[0]*100,2)
perc_fraud= np.round(cl_sup_0[1]/cl_sup_0[0]*100,2)

msg = "{} % of the dataset has a contentLength=0\n{} % has the label isHiddenFraudulent"
print(msg.format(perc_cl_sup_0, perc_fraud))

15.84 % of the dataset has a contentLength=0
25.88 % has the label isHiddenFraudulent


Interesting for the choice of features later on...
We could even create a rule based algorithm from those observations